# Data Ecosystems: Draft 1 #

The examples below provide demonstrations of a few basic DCS tasks in a git-based open source environment.

In [2]:
import pandas as pd
import numpy as np
import wbgapi as wb
import decispy as dec

pd.options.display.float_format = None

# initialize the repository to the current directory
repo = dec.git.Session()

In [3]:
# Series are loaded like this
repo.load('SP.POP.TOTL')

time    economy
YR1960  ABW           54208.0
        AFG         8996967.0
        AGO         5454938.0
        ALB         1608800.0
        AND           13410.0
                      ...    
YR2020  XKX         1775378.0
        YEM        29825968.0
        ZAF        59308690.0
        ZMB        18383956.0
        ZWE        14862927.0
Name: value, Length: 13237, dtype: float64

In [4]:
# or like this
df = repo.load(['SP.POP.TOTL', 'NY.GDP.MKTP.CD'])
df

SP.POP.TOTL  NY.GDP.MKTP.CD
time   economy                             
YR1960 ABW          54208.0             NaN
       AFG        8996967.0    5.377778e+08
       AGO        5454938.0             NaN
       ALB        1608800.0             NaN
       AND          13410.0             NaN
...                     ...             ...
YR2020 XKX        1775378.0    7.716925e+09
       YEM       29825968.0             NaN
       ZAF       59308690.0    3.354421e+11
       ZMB       18383956.0    1.811063e+10
       ZWE       14862927.0    1.805117e+10

[13237 rows x 2 columns]

In [5]:
# pandas' xs function provides easy access to individual elements
# in a MultiIndex, for example, a single year or economy
df.xs('CAN', level='economy')

,SP.POP.TOTL,NY.GDP.MKTP.CD
time,,
YR1960,17909009.0,4.046172e+10
YR1961,18271000.0,4.093495e+10
YR1962,18614000.0,4.222745e+10
YR1963,18964000.0,4.502999e+10
YR1964,19325000.0,4.937752e+10
...,...,...
YR2016,36109487.0,1.527995e+12
YR2017,36545295.0,1.649266e+12
YR2018,37065178.0,1.721853e+12


## Derivations and Aggregations ##

In [6]:
# Derivations are simple DataFrame operations
df = repo.load(['SP.POP.TOTL', 'EN.ATM.CO2E.KT'])
df['EN.ATM.CO2E.PC'] = df['EN.ATM.CO2E.KT'] * 1000 / df['SP.POP.TOTL']
df

SP.POP.TOTL  EN.ATM.CO2E.KT  EN.ATM.CO2E.PC
time   economy                                             
YR1960 ABW          54208.0       11092.675      204.631696
       AFG        8996967.0         414.371        0.046057
       AGO        5454938.0         550.050        0.100835
       ALB        1608800.0        2024.184        1.258195
       AND          13410.0             NaN             NaN
...                     ...             ...             ...
YR2020 XKX        1775378.0             NaN             NaN
       YEM       29825968.0             NaN             NaN
       ZAF       59308690.0             NaN             NaN
       ZMB       18383956.0             NaN             NaN
       ZWE       14862927.0             NaN             NaN

[13237 rows x 3 columns]

In [7]:
# For aggregations, we need region and/or income group definitions
economies = pd.read_csv('economies.csv').set_index('id')

agg = df.join(economies['region'], on='economy')
agg

SP.POP.TOTL  EN.ATM.CO2E.KT  EN.ATM.CO2E.PC region
time   economy                                                    
YR1960 ABW          54208.0       11092.675      204.631696    LCN
       AFG        8996967.0         414.371        0.046057    SAS
       AGO        5454938.0         550.050        0.100835    SSF
       ALB        1608800.0        2024.184        1.258195    ECS
       AND          13410.0             NaN             NaN    ECS
...                     ...             ...             ...    ...
YR2020 XKX        1775378.0             NaN             NaN    ECS
       YEM       29825968.0             NaN             NaN    MEA
       ZAF       59308690.0             NaN             NaN    SSF
       ZMB       18383956.0             NaN             NaN    SSF
       ZWE       14862927.0             NaN             NaN    SSF

[13237 rows x 4 columns]

In [8]:
# we also need, for reasons that will become apparent further down,
# a modified population column with values only for points that also
# have CO2 estimates
agg['POP_FOR_CO2'] = agg['SP.POP.TOTL']
agg.loc[agg['EN.ATM.CO2E.KT'].isna(), 'POP_FOR_CO2'] = np.nan

# and a dummy variable
agg['N'] = 1

agg

SP.POP.TOTL  EN.ATM.CO2E.KT  EN.ATM.CO2E.PC region  \
time   economy                                                       
YR1960 ABW          54208.0       11092.675      204.631696    LCN   
       AFG        8996967.0         414.371        0.046057    SAS   
       AGO        5454938.0         550.050        0.100835    SSF   
       ALB        1608800.0        2024.184        1.258195    ECS   
       AND          13410.0             NaN             NaN    ECS   
...                     ...             ...             ...    ...   
YR2020 XKX        1775378.0             NaN             NaN    ECS   
       YEM       29825968.0             NaN             NaN    MEA   
       ZAF       59308690.0             NaN             NaN    SSF   
       ZMB       18383956.0             NaN             NaN    SSF   
       ZWE       14862927.0             NaN             NaN    SSF   

                POP_FOR_CO2  N  
time   economy                  
YR1960 ABW          54208.0  1  
       AFG        8996967.0  1  
       AGO        5454938.0  1  
       ALB        1608800.0  1  
       AND              NaN  1  
...                     ... ..  
YR2020 XKX              NaN  1  
       YEM              NaN  1  
       ZAF              NaN  1  
       ZMB              NaN  1  
       ZWE              NaN  1  

[13237 rows x 6 columns]

In [9]:
# aggregated CO2 emissions are a simple sum. 'min_count' is necessary so that 
# sums of all NaNs produce NaN not 0
a1  = agg.groupby(['time', 'region']).sum(min_count=1)
a1

SP.POP.TOTL  EN.ATM.CO2E.KT  EN.ATM.CO2E.PC   POP_FOR_CO2   N
time   region                                                                
YR1960 EAS     1.030972e+09     1202974.018       42.636509  1.030227e+09  37
       ECS     6.667534e+08     3166989.882      158.024752  4.688119e+08  58
       LCN     2.198288e+08      297987.754      241.661475  2.172929e+08  42
       MEA     1.052032e+08       98634.966       49.757530  1.046515e+08  21
       NAC     1.986244e+08     3083748.982       30.322000  1.986244e+08   3
...                     ...             ...             ...           ...  ..
YR2020 LCN     6.522763e+08             NaN             NaN           NaN  42
       MEA     4.645541e+08             NaN             NaN           NaN  21
       NAC     3.675533e+08             NaN             NaN           NaN   3
       SAS     1.856882e+09             NaN             NaN           NaN   8
       SSF     1.132500e+09             NaN             NaN           NaN  48

[427 rows x 5 columns]

In [10]:
# WLD could also be a sum or provided separately (the second line creates a region/time index)
wld = agg.groupby('time').sum(min_count=1)
wld = wld.assign(region='WLD').set_index('region', append=True)

# add WLD to the other regions
a1 = a1.append(wld).sort_index()
a1

SP.POP.TOTL  EN.ATM.CO2E.KT  EN.ATM.CO2E.PC   POP_FOR_CO2    N
time   region                                                                 
YR1960 EAS     1.030972e+09     1202974.018       42.636509  1.030227e+09   37
       ECS     6.667534e+08     3166989.882      158.024752  4.688119e+08   58
       LCN     2.198288e+08      297987.754      241.661475  2.172929e+08   42
       MEA     1.052032e+08       98634.966       49.757530  1.046515e+08   21
       NAC     1.986244e+08     3083748.982       30.322000  1.986244e+08    3
...                     ...             ...             ...           ...  ...
YR2020 MEA     4.645541e+08             NaN             NaN           NaN   21
       NAC     3.675533e+08             NaN             NaN           NaN    3
       SAS     1.856882e+09             NaN             NaN           NaN    8
       SSF     1.132500e+09             NaN             NaN           NaN   48
       WLD     7.734490e+09             NaN             NaN           NaN  217

[488 rows x 5 columns]

In [12]:
# Some indicator aggregates are suppressed if they don't meet certain threshholds, e.g.
# values for 60% of countries. In pandas these can be masked out post-hoc
mask_values = agg.groupby(['time', 'region']).count()

# same logic as above for WLD: could be calculated or appended from external source
wld = agg.groupby('time').count().assign(region='WLD').set_index('region', append=True)
mask_values = mask_values.append(wld).sort_index()

mask_values[['EN.ATM.CO2E.KT', 'N']]

EN.ATM.CO2E.KT    N
time   region                     
YR1960 EAS                 27   37
       ECS                 28   58
       LCN                 35   42
       MEA                 19   21
       NAC                  3    3
...                       ...  ...
YR2020 MEA                  0   21
       NAC                  0    3
       SAS                  0    8
       SSF                  0   48
       WLD                  0  217

[488 rows x 2 columns]

In [13]:
# set aggregate values that don't meet our 60% threshold to NaN
a1.loc[(mask_values['EN.ATM.CO2E.KT']/mask_values['N'])<0.6, 'EN.ATM.CO2E.KT'] = np.nan
a1

SP.POP.TOTL  EN.ATM.CO2E.KT  EN.ATM.CO2E.PC   POP_FOR_CO2    N
time   region                                                                 
YR1960 EAS     1.030972e+09     1202974.018       42.636509  1.030227e+09   37
       ECS     6.667534e+08             NaN      158.024752  4.688119e+08   58
       LCN     2.198288e+08      297987.754      241.661475  2.172929e+08   42
       MEA     1.052032e+08       98634.966       49.757530  1.046515e+08   21
       NAC     1.986244e+08     3083748.982       30.322000  1.986244e+08    3
...                     ...             ...             ...           ...  ...
YR2020 MEA     4.645541e+08             NaN             NaN           NaN   21
       NAC     3.675533e+08             NaN             NaN           NaN    3
       SAS     1.856882e+09             NaN             NaN           NaN    8
       SSF     1.132500e+09             NaN             NaN           NaN   48
       WLD     7.734490e+09             NaN             NaN           NaN  217

[488 rows x 5 columns]

In [14]:
# Obviously that per capita variable is incorrect now, but easily fixed with
# the modified population variable

a1['EN.ATM.CO2E.PC'] = a1['EN.ATM.CO2E.KT'] * 1000 / a1['POP_FOR_CO2']
a1

SP.POP.TOTL  EN.ATM.CO2E.KT  EN.ATM.CO2E.PC   POP_FOR_CO2    N
time   region                                                                 
YR1960 EAS     1.030972e+09     1202974.018        1.167679  1.030227e+09   37
       ECS     6.667534e+08             NaN             NaN  4.688119e+08   58
       LCN     2.198288e+08      297987.754        1.371365  2.172929e+08   42
       MEA     1.052032e+08       98634.966        0.942509  1.046515e+08   21
       NAC     1.986244e+08     3083748.982       15.525529  1.986244e+08    3
...                     ...             ...             ...           ...  ...
YR2020 MEA     4.645541e+08             NaN             NaN           NaN   21
       NAC     3.675533e+08             NaN             NaN           NaN    3
       SAS     1.856882e+09             NaN             NaN           NaN    8
       SSF     1.132500e+09             NaN             NaN           NaN   48
       WLD     7.734490e+09             NaN             NaN           NaN  217

[488 rows x 5 columns]

## Updates and External Ingestion ##

New or updated data would need to be keyed by economy, time/economy, or
series/time/economy. This could come a variety of sources ranging from manually entered
to APIs or web scraping

For example, % of Parliamentary Seats Held by Women (SG.GEN.PARL.ZS) is sourced from
IPU: <https://data.ipu.org/women-ranking>.

In [15]:
# start by loading the data to be updated
df = repo.load('SG.GEN.PARL.ZS')
df

time    economy
YR1960  ABW              NaN
        AFG              NaN
        AGO              NaN
        ALB              NaN
        AND              NaN
                     ...    
YR2020  XKX              NaN
        YEM         0.332226
        ZAF        46.750000
        ZMB        16.766467
        ZWE        31.851852
Name: value, Length: 13237, dtype: float64

In [16]:
# Here is the URL, accessed from the link above for CSV access
# (with a token inserted for the year)
url = 'https://data.ipu.org/api/women-ranking.csv?month=12&year={}&load-entity-refs=taxonomy_term%2Cfield_collection_item&max-depth=2&langcode=en'

# we can read this CSV straight from the internet with parameters to skip krufty data
# correctly interpret hyphens as NaN, and apply useful column names
col_names = ['rank', 'country', 'date', 'seats', 'women', 'women_pct']
ipu = pd.read_csv(url.format(2020), skiprows=6, names=col_names, index_col=False, na_values='-')
ipu

,rank,country,date,seats,women,women_pct
0,1,Rwanda,9.2018,80.0,49.0,61.3
1,2,Cuba,3.2018,605.0,322.0,53.2
2,3,United Arab Emirates,10.2019,40.0,20.0,50.0
3,4,New Zealand,10.2020,120.0,58.0,48.3
4,5,Mexico,7.2018,500.0,241.0,48.2
...,...,...,...,...,...,...
188,“,Papua New Guinea,6.2017,111.0,0.0,0.0
189,“,Vanuatu,3.2020,52.0,0.0,0.0
190,NaN,Eritrea,2.1994,NaN,NaN,NaN
191,NaN,Haiti,8.2015,NaN,NaN,NaN


In [17]:
# cleaning: we need ISO codes not country names, along with a time dimension
# the wbgapi module includes a utility to code common country names, especially
# from the UN system

ipu['economy'] = wb.economy.coder(ipu['country'])
ipu['time'] = 'YR2020'
ipu.set_index(['time', 'economy'], inplace=True)
ipu

rank               country     date  seats  women  \
time   economy                                                           
YR2020 RWA              1                Rwanda   9.2018   80.0   49.0   
       CUB              2                  Cuba   3.2018  605.0  322.0   
       ARE              3  United Arab Emirates  10.2019   40.0   20.0   
       NZL              4           New Zealand  10.2020  120.0   58.0   
       MEX              5                Mexico   7.2018  500.0  241.0   
...                   ...                   ...      ...    ...    ...   
       PNG              “      Papua New Guinea   6.2017  111.0    0.0   
       VUT              “               Vanuatu   3.2020   52.0    0.0   
       ERI            NaN               Eritrea   2.1994    NaN    NaN   
       HTI            NaN                 Haiti   8.2015    NaN    NaN   
       SDN      Suspended                 Sudan      NaN    NaN    NaN   

                women_pct  
time   economy             
YR2020 RWA           61.3  
       CUB           53.2  
       ARE           50.0  
       NZL           48.3  
       MEX           48.2  
...                   ...  
       PNG            0.0  
       VUT            0.0  
       ERI            NaN  
       HTI            NaN  
       SDN            NaN  

[193 rows x 6 columns]

In [18]:
# There are various approaches for updating new data in the master data frame.
# The simplest would be to drop previous 2020 values and replace them with
# the ones from IPU. However, they only provide values for 193 countries out
# of our 217, so that simple approach would drop 24 empty rows with possibly
# unpleasant side effects

# To maintain integrity while keeping the code simple, we start with an empty
# data frame, which we update with IPU data, giving us the proper number of economies
new_data = repo.Series(None, 'YR2020')
new_data.update(ipu['women_pct'])
new_data

time    economy
YR2020  ABW         NaN
        AFG        27.0
        AGO        30.0
        ALB        29.5
        AND        46.4
                   ... 
        XKX         NaN
        YEM         0.3
        ZAF        46.8
        ZMB        16.8
        ZWE        31.9
Length: 217, dtype: float64

In [19]:
# Now we can safely and simply drop the previous values in the master data frame,
# append the new values, and save to the working directory

df = df.drop('YR2020', level='time', errors='ignore').append(new_data)
repo.save(df, 'SG.GEN.PARL.ZS')

# NB: this cell changes file contents in the working directory

## Validation and Management ##

In [20]:
# We use the git package to see if the previous update resulted in any real changes.
# this function shows a list of indicators that have changed since the last commit
repo.changes()

['SG.GEN.PARL.ZS']

In [21]:
# Here, we load a copy of the indicator from the working directory and from the
# last commit to compare them
a = repo.load('SG.GEN.PARL.ZS')
b = repo.load('SG.GEN.PARL.ZS', ref='HEAD')
a.compare(b)

self      other
time   economy                 
YR2020 AFG      27.0  27.016129
       ALB      29.5  29.508197
       AND      46.4  46.428571
       ARG      40.9  40.856031
       ARM      23.5  23.484848
...              ...        ...
       VNM      26.7  26.720648
       YEM       0.3   0.332226
       ZAF      46.8  46.750000
       ZMB      16.8  16.766467
       ZWE      31.9  31.851852

[158 rows x 2 columns]

In [22]:
# this comparison raises the question if our methodology is consistent. The raw IPU
# data is rounded to 1 decimal point but the previous commit includes at least 6 decimal
# points precision. We should confirm with Haruna whether she calculated the percentage
# or took the published value, but let's assume she calculated it and save the data again

ipu['women_pct2'] = ipu['women'] * 100 / ipu['seats']
new_data.update(ipu['women_pct2'])
df = df.drop('YR2020', level='time', errors='ignore').append(new_data)
repo.save(df, 'SG.GEN.PARL.ZS')

# and then try the comparison again. This time showing greater precision
pd.options.display.float_format = '{:.15f}'.format
a = repo.load('SG.GEN.PARL.ZS')
a.compare(b)

self              other
time   economy                                      
YR2020 AFG     27.016129032258064 27.016129032258100
       ALB     29.508196721311474 29.508196721311499
       AND     46.428571428571431 46.428571428571402
       ARG     40.856031128404666 40.856031128404702
       ARM     23.484848484848484 23.484848484848499
...                           ...                ...
       VEN     22.155688622754489 22.155688622754500
       VNM     26.720647773279357 26.720647773279399
       YEM      0.332225913621262  0.332225913621262
       ZMB     16.766467065868262 16.766467065868301
       ZWE     31.851851851851851 31.851851851851901

[141 rows x 2 columns]

In [23]:
# less differences but still reporting quite a few differences which look like
# mostly floating point errors. Let's try rounding to, say 6 decimal points 
# to find just the significant differences
a = np.round(a, 6)
b = np.round(b, 6)

# For readability we'll add back in the country names
a.compare(b).join(ipu['country'])

self              other    country
time   economy                                                 
YR2020 AUS     30.000000000000000 30.463576000000000  Australia
       NGA      3.611111000000000  7.222222000000000    Nigeria